In [1]:
import os
import pickle
import requests
import numpy as np
import random
import re

In [2]:
x = 128
def list_to_string(a):
    a = str(a)
    return a.replace(' ', '')

def num_to_list(num):
    return [int(x) for x in str(num)]

list_x = num_to_list(x)
print(list_x)
print(list_to_string(list_x))

[1, 2, 8]
[1,2,8]


In [3]:
def get_input_string(x,y):
    x, y = str(x), str(y)
    input_str = f'Input:\n{x}+{y}\n'

    return input_str

def get_output_string(x,y):
    x, y = str(x), str(y)

    len_x, len_y = len(x), len(y)
    list_x, list_y = num_to_list(x), num_to_list(y)

    output_str = f'Target:\n<scratch>\n'

    output_str += f'{list_to_string(list_x)} has {len_x} digits.\n'
    output_str += f'{list_to_string(list_y)} has {len_y} digits.\n'

    C=0
    A=[]
    for i in range(max(len_x, len_y)):
        a = list_x[-1] if i < len_x else 0
        b = list_y[-1] if i < len_y else 0
        c = a + b + C
        
        output_str += f'{list_to_string(list_x)} + {list_to_string(list_y)} , A={list_to_string(A)} , C={C} , {a}+{b}+{C}={c} , A->{c%10} , C->{c//10}\n'

        A.insert(0, c % 10)
        C = c // 10

        list_x = list_x[:-1]
        list_y = list_y[:-1]

    output_str += f'{list_to_string(list_x)} + {list_to_string(list_y)} , A={list_to_string(A)} C={C} , END\n</scratch>\n'
    if C == 1:
        A.insert(0, 1)
    for a in A:
        output_str += f'{a} '

    return output_str[:-1]+'\n'

In [4]:
x,y = 128, 367
input_str = get_input_string(x,y)
output_str = get_output_string(x,y)

print(input_str+output_str)
print(len(input_str+output_str))

Input:
128+367
Target:
<scratch>
[1,2,8] has 3 digits.
[3,6,7] has 3 digits.
[1,2,8] + [3,6,7] , A=[] , C=0 , 8+7+0=15 , A->5 , C->1
[1,2] + [3,6] , A=[5] , C=1 , 2+6+1=9 , A->9 , C->0
[1] + [3] , A=[9,5] , C=0 , 1+3+0=4 , A->4 , C->0
[] + [] , A=[4,9,5] C=0 , END
</scratch>
4 9 5

282


In [9]:
num_samples = 4000

input_file_path = f'original_balanced_data/add_examples_{num_samples}_shuffled.txt'
output_file_path = f'add_examples_algorithmic_{num_samples}.txt'

with open(input_file_path, 'r') as f:
    lines = f.readlines()

with open(output_file_path, 'w') as f:
    for line in lines:
        x,y = re.split('\$|\+|=',line)[1:3]
        x, y = int(x), int(y)
        input_str = get_input_string(x,y)
        output_str = get_output_string(x,y)
        f.write(input_str+output_str)


In [10]:
input_file_path = f'add_examples_algorithmic_{num_samples}.txt'

with open(input_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")

# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers
def decode(l):
    ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# create the train and test splits
# n = len(data) # 130,023
# train_data = data[:int(n*0.9)]
# val_data = data[int(n*0.9):]

# encode both to integers
train_ids = encode(data)
# val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
# print(f"val has {len(val_ids):,} tokens")

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
# val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(f'train_balanced_{num_samples}.bin')
# val_ids.tofile(f'val_balanced.bin')

# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open(f'meta.pkl', 'wb') as f:
    pickle.dump(meta, f)

length of dataset in characters: 1,088,520
all the unique characters: 
 +,-./0123456789:<=>ACDEINT[]acdeghinprstu
vocab size: 43
train has 1,088,520 tokens


In [ ]:
input_file_path = 'add_examples_balanced.txt'
output_file_path = 'prompt_train.txt'
